<h1> <center> Big Mac Index - Panel Data Analysis Project </center> </h1>
<center> <h2>  1. Data Pre-processing </h2> </center>
<br></br>

In [1]:
# Import libraries
import pandas as pd

**All Data Upload**

In [7]:
# Load contraints

country_list = pd.read_excel('constraints.xlsx', sheet_name = 0)
years = pd.read_excel('constraints.xlsx', sheet_name = 1)
years['year'] = years['year'].astype('str')

# Load datasets
big_mac = pd.read_csv('big-mac-raw-index.csv', usecols = ['date', 'iso_a3', 'local_price', 'dollar_ex'])
gdp_data = pd.read_csv('API_NY.GDP.PCAP.PP.CD_DS2_en_csv_v2_4150885\API_NY.GDP.PCAP.PP.CD_DS2_en_csv_v2_4150885.csv')
acc_balance = pd.read_csv('API_BN.CAB.XOKA.GD.ZS_DS2_en_csv_v2_4150849\API_BN.CAB.XOKA.GD.ZS_DS2_en_csv_v2_4150849.csv')
labcomp = pd.read_excel('pwt100.xlsx', sheet_name = 2)
ggb = pd.read_csv('DP_LIVE_28052022104927583.csv')

TypeError: Passing a bool to header is invalid. Use header=None for no header or header=int or list-like of ints to specify the row(s) making up the column names

**Big Mac Index Pre-processing**

In [ ]:
# Filter Country List
big_mac_country_filtered = pd.merge(left = country_list['iso_a3'], 
                                    right = big_mac, 
                                    how = 'left', 
                                    on = 'iso_a3')

# Group to annual
big_mac_country_filtered['Year'] = big_mac_country_filtered['date'].str[:4]
big_mac_annual = big_mac_country_filtered.groupby(by = ['iso_a3', 'Year'], axis = 0, 
                                                  as_index = False).agg({'local_price': 'mean', 'dollar_ex': 'mean'})

# Filter Timeperiod
big_mac_annual = pd.merge(left = years, 
                          right = big_mac_annual, 
                          how = 'left', 
                          left_on = 'year',
                          right_on = 'Year')

# Create Big Mac Index
big_mac_annual['dollar_price'] = big_mac_annual['local_price'] / big_mac_annual['dollar_ex']
big_mac_usa = big_mac_annual.loc[big_mac_annual['iso_a3'] == 'USA',['Year', 'local_price']]
big_mac_usa.rename(columns = {'local_price':'price_in_USA'}, inplace = True)
big_mac_annual = pd.merge(left = big_mac_annual, right = big_mac_usa, on = 'Year')
big_mac_annual['big_mac_index'] = (big_mac_annual['dollar_price'] / big_mac_annual['price_in_USA']) - 1
big_mac_annual.drop('price_in_USA', axis = 1, inplace = True)

# Export
#big_mac_annual.to_csv('big_mac_annual.csv')

**GDP Per Capita Pre-processing**

In [ ]:
# Filter Country List
gdp_data['Country Code'] = gdp_data['Country Code'].apply(lambda x: 'EUZ' if x == 'EMU' else x)
gdp_data_country_filtered = pd.merge(left = country_list['iso_a3'], 
                                     right = gdp_data, 
                                     how = 'left', 
                                     left_on = 'iso_a3', 
                                     right_on = 'Country Code')
# Drop Columns
drop_cols = ['Indicator Name', 'Indicator Code']
gdp_data_country_filtered.drop(drop_cols, axis = 1, inplace = True)

# Filter Timeperiod
keep_cols = ['Country Name', 'Country Code', 'iso_a3']
keep_cols.extend(list(years.year))
gdp_data_year_filtered = gdp_data_country_filtered[keep_cols]

# Wide to Long Format
gdp_post_processed = pd.melt(frame = gdp_data_year_filtered, 
                             id_vars = ['Country Name', 'Country Code', 'iso_a3'], 
                             var_name = 'Year', 
                             value_name = 'GDP per capita')

# Export
#gdp_post_processed.to_csv('gdp_post_processed.csv')

**Current Account Balance Pre-processing**

In [ ]:
# Filter Country List
acc_balance['Country Code'] = acc_balance['Country Code'].apply(lambda x: 'EUZ' if x == 'EMU' else x)
acc_balance_country_filtered = pd.merge(left = country_list['iso_a3'], 
                                        right = acc_balance, 
                                        how = 'left', 
                                        left_on = 'iso_a3', 
                                        right_on = 'Country Code')
# Drop Columns
drop_cols = ['Indicator Name', 'Indicator Code']
acc_balance_country_filtered.drop(drop_cols, axis = 1, inplace = True)

# Filter Timeperiod
keep_cols = ['Country Name', 'Country Code', 'iso_a3']
keep_cols.extend(list(years.year))
acc_balance_year_filtered = acc_balance_country_filtered[keep_cols]

# Wide to Long Format
acc_balance_post_processed = pd.melt(frame = acc_balance_year_filtered, 
                                     id_vars = ['Country Name', 'Country Code', 'iso_a3'], 
                                     var_name = 'Year', 
                                     value_name = 'Curr Acc Balance')

# Export
acc_balance_post_processed.to_csv('acc_balance_post_processed.csv')

**Labour Compensation Dataset Pre-processing**

In [ ]:
# Filter Country List
labcomp['new_countrycode'] = labcomp.apply(lambda x: 'EUZ' if x['currency_unit'] == 'Euro' else x['countrycode'], axis = 1)
labcomp_data_country_filtered = pd.merge(left = country_list, 
                                         right = labcomp, 
                                         how = 'left', 
                                         left_on = 'iso_a3', 
                                         right_on = 'new_countrycode')
keep_cols = ['name', 'countrycode', 'country', 'year', 'labsh', 'new_countrycode']
labcomp_data_country_filtered = labcomp_data_country_filtered[keep_cols]

# Average labor compensation for all european countries
eur = labcomp_data_country_filtered[labcomp_data_country_filtered['new_countrycode'] == 'EUZ']
eur = eur.groupby(['year'], as_index=False).mean()
eur['name'] = 'Euro Area'
eur['countrycode'] = 'EUZ'

# Remove european data from labor compensation dataframe & append average european numbers
labcomp_data_country_filtered.drop(columns=['countrycode','country'], inplace=True)
labcomp_data_country_filtered = labcomp_data_country_filtered.rename(columns={'new_countrycode':'countrycode'})
labcomp_data_country_filtered = labcomp_data_country_filtered[labcomp_data_country_filtered['countrycode'] != 'EUZ']
labcomp_data_country_filtered = labcomp_data_country_filtered.append(eur)

# Filter Time-period
labcomp_data_country_filtered = labcomp_data_country_filtered[(labcomp_data_country_filtered['year'] >= 2000) 
                                                              & (labcomp_data_country_filtered['year'] <= 2016)]
labcomp_data_country_filtered['year'] = labcomp_data_country_filtered['year'].astype('str')


# Rename
lab_comp_post_processed = labcomp_data_country_filtered.rename(columns = {'name':'Country Name', 
                                                                          'year':'Year', 
                                                                          'countrycode': 'Country Code',
                                                                          'labsh': 'labor comp per capita'})

# Export
#lab_comp_post_processed.to_csv('lab_comp_post_processed.csv')

**Government Balance Dataset Preprocessing**

In [ ]:
# Filter country list
ggb['LOCATION'] = ggb['LOCATION'].apply(lambda x: 'EUZ' if x == 'EA' else x)
ggb_data_country_filtered = pd.merge(left = country_list, 
                                    right = ggb, 
                                    how = 'left', 
                                    left_on = 'iso_a3', 
                                    right_on = 'LOCATION')

# Filter time-period
ggb_data_country_filtered = ggb_data_country_filtered[(ggb_data_country_filtered['TIME'] >= 2000) & 
                                                      (ggb_data_country_filtered['TIME'] <= 2016)]
ggb_data_country_filtered['TIME'] = ggb_data_country_filtered['TIME'].astype('str')
ggb_data_country_filtered['TIME'] = ggb_data_country_filtered['TIME'].str[:4]

# Drop & rename columns
ggb_data_country_filtered.drop(columns=['INDICATOR','SUBJECT','MEASURE','FREQUENCY','Flag Codes','currency_code',
                                        'iso_a3'], inplace = True)
ggb_data_post_processed = ggb_data_country_filtered.rename(columns={'name':'Country Name', 
                                                                    'TIME':'Year', 
                                                                    'LOCATION': 'Country Code', 
                                                                    'Value': 'GGB'})

# Export
#ggb_data_post_processed.to_csv('ggb_data_post_processed.csv')

**Merge data**

In [ ]:
# Merge GDP & Current Account Balance
merge1 = pd.merge(left = gdp_post_processed,
                  right = acc_balance_post_processed[['iso_a3', 'Year', 'Curr Acc Balance']],
                  how = 'inner', 
                  on = ['iso_a3', 'Year'])

# Merge GGP & Labor Compensation
merge2 = pd.merge(left = ggb_data_post_processed,
                  right = lab_comp_post_processed[['Year', 'Country Code', 'labor comp per capita']],
                  how = 'right', 
                  on = ['Country Code', 'Year'])

# Missing value Imputation
merge2['GGB'] = merge2['GGB'].fillna(merge2.groupby(by = 'Country Code')['GGB'].transform('mean'))

# Merge with Big Mac Index
merge3 = pd.merge(left = big_mac_annual,
                  right = merge1[['iso_a3', 'Year', 'GDP per capita', 'Curr Acc Balance']],
                  how = 'left',
                  on = ['iso_a3', 'Year'])

merge4 = pd.merge(left = merge3,
                  right = merge2[['Country Code', 'Year', 'GGB', 'labor comp per capita']],
                  how = 'left',
                  left_on = ['iso_a3', 'Year'],
                  right_on = ['Country Code', 'Year'])

# Export Data
#merge4.to_csv('processed_data_I.csv')